In [1]:
import os

In [ ]:
openai_api_key = ""

In [4]:
from langchain_openai import OpenAIEmbeddings
embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [6]:
from langchain_community.document_loaders import TextLoader

In [7]:
from pathlib import Path

In [8]:
docs_base = Path("./docs")
files_all = docs_base.rglob("*")
excludes = [".DS_Store", ".ipynb_checkpoints"]
files = []
for f in files_all:
    to_exclude = False
    for ex in excludes:
        to_exclude = to_exclude or f.match(ex)
    if not to_exclude and f.is_file():
        files.append(f)

In [9]:
files

[PosixPath('docs/menu_price.csv'),
 PosixPath('docs/general_info.txt'),
 PosixPath('docs/coffee_bean_menu.csv'),
 PosixPath('docs/product_factsheets/geisha.txt'),
 PosixPath('docs/product_factsheets/blue_mountain.txt'),
 PosixPath('docs/product_factsheets/sumatra_mandheling.txt'),
 PosixPath('docs/product_factsheets/yirgacheffe.txt')]

In [10]:
documents = []
for file in files:
    print(file)
    loader = TextLoader(str(file.absolute()))
    documents += loader.load()

docs/menu_price.csv
docs/general_info.txt
docs/coffee_bean_menu.csv
docs/product_factsheets/geisha.txt
docs/product_factsheets/blue_mountain.txt
docs/product_factsheets/sumatra_mandheling.txt
docs/product_factsheets/yirgacheffe.txt


In [11]:
documents[0].__dict__

{'id': None,
 'metadata': {'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/menu_price.csv'},
 'page_content': '\ufeffTable of Drink Menu,,\nCoffee Drink,Small Size Price ($),Large Size Price ($)\nEspresso,2.00,3.00\nAmericano,2.50,3.50\nCappuccino,3.00,4.50\nLatte,3.50,5.00\nMocha,4.00,5.50\n,,\nTable of Add-on items or change,,\nIce,0.50,\nOat milk,1.00,\nSoy milk,0.75,',
 'type': 'Document'}

Indexing with Chroma

In [12]:
from langchain_community.vectorstores import Chroma

In [13]:
db = Chroma.from_documents(documents, embedding_llm, persist_directory="./chroma",
                          collection_name="planetbucks")

In [14]:
db.persist()

/var/folders/b8/k4y3h5991wjcdwnm2lfrm09h0000gn/T/ipykernel_3954/123899826.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


Retrival

In [15]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate


Create Embedding LLM (OpenAI)

In [16]:
embedding_llm = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [17]:
db = Chroma(persist_directory='./chroma', embedding_function=embedding_llm,
            collection_name="planetbucks")

/var/folders/b8/k4y3h5991wjcdwnm2lfrm09h0000gn/T/ipykernel_3954/3644769364.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory='./chroma', embedding_function=embedding_llm,


In [18]:
query = "opening hours"
result = db.similarity_search_with_relevance_scores(query)
print(len(result))

4


In [19]:
result

[(Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/general_info.txt'}, page_content='Shop Name: PlanetBucks\nWhat we sell: coffee drinks and coffee bean\n\nOperating time:\nMonday to Friday 7 am - 8 pm\nSaturday, Sunday, National Holiday 9 am - 4 pm\n\nAddress: ชั้น G ห้าง Emquartier สุขุมวิท กรุงเทพฯ\nBranch 1: Icon Siam Mall, Third Floor, South Wing, Bangkok, Thailand\nBranch 2: 103th Floor, Empire State Building, NYC\n\nPayment: cash, credit card, square, bitcoin'),
  0.7053169136827597),
 (Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/menu_price.csv'}, page_content='\ufeffTable of Drink Menu,,\nCoffee Drink,Small Size Price ($),Large Size Price ($)\nEspresso,2.00,3.00\nAmericano,2.50,3.50\nCappuccino,3.00,4.50\nLatte,3.50,5.00\nMocha,4.00,5.50\n,,\nTable of Add-on items or change,,\nIce,0.50,\nOat milk,1.00,\nSoy milk,0.75,'),
  0.6431774190226602),
 (Document(

In [20]:
result = db.similarity_search_with_relevance_scores(query, k=2)
print(len(result))

2


In [21]:
result

[(Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/general_info.txt'}, page_content='Shop Name: PlanetBucks\nWhat we sell: coffee drinks and coffee bean\n\nOperating time:\nMonday to Friday 7 am - 8 pm\nSaturday, Sunday, National Holiday 9 am - 4 pm\n\nAddress: ชั้น G ห้าง Emquartier สุขุมวิท กรุงเทพฯ\nBranch 1: Icon Siam Mall, Third Floor, South Wing, Bangkok, Thailand\nBranch 2: 103th Floor, Empire State Building, NYC\n\nPayment: cash, credit card, square, bitcoin'),
  0.7053169136827597),
 (Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/menu_price.csv'}, page_content='\ufeffTable of Drink Menu,,\nCoffee Drink,Small Size Price ($),Large Size Price ($)\nEspresso,2.00,3.00\nAmericano,2.50,3.50\nCappuccino,3.00,4.50\nLatte,3.50,5.00\nMocha,4.00,5.50\n,,\nTable of Add-on items or change,,\nIce,0.50,\nOat milk,1.00,\nSoy milk,0.75,'),
  0.6431774190226602)]

MMR (Maximum marginal relevance)
MMR = iteratively find documents that are dissimilar to previous results.
It could improve performance for retrievals.

In [22]:
db.max_marginal_relevance_search(query, k=2)

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


[Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/general_info.txt'}, page_content='Shop Name: PlanetBucks\nWhat we sell: coffee drinks and coffee bean\n\nOperating time:\nMonday to Friday 7 am - 8 pm\nSaturday, Sunday, National Holiday 9 am - 4 pm\n\nAddress: ชั้น G ห้าง Emquartier สุขุมวิท กรุงเทพฯ\nBranch 1: Icon Siam Mall, Third Floor, South Wing, Bangkok, Thailand\nBranch 2: 103th Floor, Empire State Building, NYC\n\nPayment: cash, credit card, square, bitcoin'),
 Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/product_factsheets/yirgacheffe.txt'}, page_content="Yirgacheffe Coffee Beans - The Essence of Ethiopia\n\nOrigin: Sourced from the esteemed Yirgacheffe region within Sidamo, Ethiopia, where coffee cultivation is an art passed down through generations.\nAltitude: Cultivated at 1,700 to 2,200 meters, the ideal range for developing its signature floral and c

In [23]:
retriever = db.as_retriever(search_type='mmr', search_kwargs={'k': 2, 'fetch_k': 5})

In [24]:
retriever.invoke(query)

[Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/general_info.txt'}, page_content='Shop Name: PlanetBucks\nWhat we sell: coffee drinks and coffee bean\n\nOperating time:\nMonday to Friday 7 am - 8 pm\nSaturday, Sunday, National Holiday 9 am - 4 pm\n\nAddress: ชั้น G ห้าง Emquartier สุขุมวิท กรุงเทพฯ\nBranch 1: Icon Siam Mall, Third Floor, South Wing, Bangkok, Thailand\nBranch 2: 103th Floor, Empire State Building, NYC\n\nPayment: cash, credit card, square, bitcoin'),
 Document(metadata={'source': '/Users/vinitpahwa/Documents/GENAIProjects/customer-chatbot-demo-langchain/docs/coffee_bean_menu.csv'}, page_content='\ufeffTopic,Geisha,Yirgacheffe,Blue Mountain,Sumatra Mandheling\nOrigin,"Hacienda La Esmeralda, Panama","Yirgacheffe region, Ethiopia","Blue Mountains, Jamaica","Mandheling, Sumatra, Indonesia"\nAltitude (meters),"1,500-2,000","1,700-2,200","910-1,700","1,000-1,500"\nFlavor Profile,"Jasmine, bergamot, honey, peaches, 

In [25]:
retriever_tool = create_retriever_tool(
    db.as_retriever(search_type='mmr'),
    name = "planetbucks_search",
    description = """Search for information about PlanetBucks store, including general information, 
    coffee drink menus, specialty coffee beans menu, and bean fact sheet.""",
)

In [26]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

In [27]:
system_message = """You are a helpful assistant working at a coffee shop.
You can use a given chat history and given tools to respond to a user.
Your character is a polite and friendly female.
You answer concisely, without introduction or appending.
If you do not know the answer, just say 'I do not know.'
When you calculate the price of an ordered item, you should think step-by-step.
"""

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_message)),
        MessagesPlaceholder(
            variable_name="chat_history", optional=True
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{input}"
        ),  # Where the human input will injected
        MessagesPlaceholder(
            variable_name="agent_scratchpad"
        ),  # Where the memory will be stored.
    ]
)

In [29]:
tools = [retriever_tool, ]

In [30]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory

agent = create_openai_functions_agent(llm, tools, prompt,)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

/var/folders/b8/k4y3h5991wjcdwnm2lfrm09h0000gn/T/ipykernel_3954/549322652.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [31]:
def wrapper_chat_history(chat_history, memory):
    chat_history = []
    for m in memory.chat_memory.messages:
        chat_history.append(m.content)
    return chat_history

In [32]:
def converse(message, chat_history):
    response = agent_executor.invoke({"input": message})
    chat_history = wrapper_chat_history(chat_history, memory)
    return response['output']

In [ ]:
import gradio as gr

demo = gr.ChatInterface(fn=converse)

demo.launch(share=False)

/opt/anaconda3/envs/chatenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/chatenv/lib/python3.10/site-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Coffee is a popular beverage made from roasted coffee beans, which are the seeds of berries from the Coffea plant. It is typically brewed by extracting flavors from the ground beans with hot water.

> Finished chain.


> Entering new AgentExecutor chain...
Coffee shops can be found in various locations such as shopping centers, downtown areas, neighborhoods, and even standalone buildings. You can use online maps or directories to locate coffee shops near you.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `planetbucks_search` with `{'query': 'best coffee bean'}`




Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags for maximum freshness.
Price: $40 per bag, a testament to its unparalleled quality and heritage.
Certifications: Officially certified as 100% Blue Mountain Coffee, ensuring authenticity and excellence.
Awards: Renowned worldwide for its exceptional q

...

Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags for maximum freshness.
Price: $40 per bag, a testament to its unparalleled quality and heritage.
Certifications: Officially certified as 100% Blue Mountain Coffee, ensuring authenticity and excellence.
Awards: Renowned worldwide for its exceptional q

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


﻿Topic,Geisha,Yirgacheffe,Blue Mountain,Sumatra Mandheling
Origin,"Hacienda La Esmeralda, Panama","Yirgacheffe region, Ethiopia","Blue Mountains, Jamaica","Mandheling, Sumatra, Indonesia"
Altitude (meters),"1,500-2,000","1,700-2,200","910-1,700","1,000-1,500"
Flavor Profile,"Jasmine, bergamot, honey, peaches, berries","Floral, lemon, bergamot, jasmine, honey","Sweet herbs, nuts, chocolate","Dark chocolate, cedar, tropical fruit"
Acidity,Bright,Vibrant,Mild,Low
Body,Delicate,Light-Medium,Smooth,Full
Aftertaste,Sweet,"Clean, sweet","Clean, chocolatey",Spicy
Processing Method,Washed,Washed,Wet-processed,Giling Basah
Roast Level,Light-Medium,Light-Medium,Medium,Medium-Dark
Recommended Brewing Methods,"Pour-over, Chemex, Aeropress","Pour-over, cold brew, drip","Drip, French press","Espresso, French press"
Packaging (oz/g),12 oz (340g),12 oz (340g),16 oz (454g),12 oz (340g)
Price ($ per bag),$50,$30,$40,$25
Certifications,"Direct Trade, Rainforest Alliance","Organic, Fair Trade",100% Certifi

Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


﻿Table of Drink Menu,,
Coffee Drink,Small Size Price ($),Large Size Price ($)
Espresso,2.00,3.00
Americano,2.50,3.50
Cappuccino,3.00,4.50
Latte,3.50,5.00
Mocha,4.00,5.50
,,
Table of Add-on items or change,,
Ice,0.50,
Oat milk,1.00,
Soy milk,0.75,

Blue Mountain Coffee Beans - The Jewel of Jamaica

Origin: Cultivated in the revered Blue Mountains of Jamaica, from estates that meet the highest standards of quality.
Altitude: Thrives at altitudes of 910 to 1,700 meters, where the climate is perfect for developing its signature flavors.
Taste Profile: A rare blend of sweet herbs and nuts, complemented by a chocolate undertone. It boasts a mild acidity, a smooth body, and a clean, sweet aftertaste.
Processing: Wet-processed to enhance its smooth and clean profile.
Roast Level: Medium roast, optimizing its mild and unique flavors.
Brewing Recommendations: Ideal for drip coffee makers and French press, where its subtle complexities can be savored.
Packaging: 16 oz (454g) vacuum-sealed bags fo